In [158]:
# imports
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances,silhouette_samples,silhouette_score
import matplotlib.pyplot as plt
import pickle

In [159]:
# reading in train data
df = pd.read_csv('data/train_preprocessed_data.csv')
pd.set_option('max_columns', None)
df = df.drop(columns=['loan_paid'])
df

,requested_amnt,annual_income,debt_to_income_ratio,public_bankruptcies,state,delinquency_last_2yrs,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,any_tax_liens,missing_vals,income_to_loan_amount_ratio,fico_score,fico_score_range,employment_length,employment_verified,type_of_application,loan_duration,reason_for_loan_credit_card,reason_for_loan_debt_consolidation,reason_for_loan_home_improvement,reason_for_loan_major_purchase,reason_for_loan_other,reason_for_loan_small_business,employment_Manager,employment_Owner,employment_RN,employment_Registered Nurse,employment_Teacher,employment_other,home_ownership_status_ANY,home_ownership_status_MORTGAGE,home_ownership_status_NONE,home_ownership_status_OTHER,home_ownership_status_OWN,home_ownership_status_RENT,zipcode_bad,zipcode_good,zipcode_mid,zipcode_mid_bad,zipcode_mid_good
0,2.001849,2.429058,-0.181194,0,15,0,0,-0.114735,0.271535,1.984340,0,1,0.128000,2.577382,4.0,4,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,-0.976558,-0.101551,-0.291395,0,3,0,1,-0.171642,-0.442526,-0.481253,0,0,0.085714,-1.196633,4.0,10,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
2,-1.182756,-0.565496,0.271310,0,7,1,0,-1.252866,-0.523397,-0.597631,0,0,0.113514,-0.743751,4.0,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
3,-0.549844,-0.453025,1.906763,0,22,0,0,-0.114735,0.237305,-0.194368,0,1,0.216111,0.463934,4.0,10,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
4,1.085416,0.123392,1.354786,0,27,0,0,2.446060,0.621609,1.943744,0,0,0.279070,-1.196633,4.0,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967914,-0.747450,-0.087492,0.753072,1,29,1,2,-0.911427,-0.062683,-0.042805,0,0,0.112676,-0.743751,4.0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
967915,-1.205666,-0.551437,-0.475712,0,15,0,0,2.502967,-0.508778,-0.635522,0,0,0.105263,-0.139909,4.0,6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
967916,-0.403788,-0.277288,-0.858001,0,11,0,1,-0.114735,-0.242844,-0.524557,0,1,0.191304,1.067776,4.0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
967917,-0.014304,-0.410848,-0.166566,0,43,0,1,-0.114735,-0.025935,-0.318865,0,1,0.300000,0.011052,4.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


In [160]:
# setting up for k-means
x = df.to_numpy()
x[np.isinf(x)] = 0
print(np.where(np.isinf(x)))

(array([], dtype=int64), array([], dtype=int64))


In [ ]:
# silhouette analysis with different k-values
k = range(3, 10)
scores3 = []
for cur_k in k:
    kmeans = KMeans(n_clusters=cur_k)
    kmeans.fit(x)
    print(kmeans.labels_)
    print(type(kmeans.labels_))
    print(np.unique(kmeans.labels_, return_counts=True))
    score = silhouette_score(x, kmeans.labels_, metric='euclidean', sample_size=25000)
    print(score)
    scores3.append(score)

In [ ]:
plt.plot(range(3, 10), scores3)

In [161]:
# training the kmeans
best_k = 10
kmeans = KMeans(best_k, random_state=42)
kmeans.fit(x)

KMeans(n_clusters=10, random_state=42)

In [162]:
# predicting and finding the cluster frequency
train_cluster_pred = kmeans.predict(x).astype(int)
temp = np.unique(train_cluster_pred, return_counts=True)
counts = dict(zip(temp[0], temp[1]))
print(counts)

{0: 154831, 1: 1, 2: 180897, 3: 1, 4: 136683, 5: 1, 6: 117618, 7: 158668, 8: 84285, 9: 134934}


In [163]:
# saving model using pickles        
with open('models/k_means.pickle', 'wb') as handle:
        pickle.dump(kmeans, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [164]:
# reading in val data
df1 = pd.read_csv('data/predict_preprocessed_data.csv')
pd.set_option('max_columns', None)
df1 = df1.drop(columns=['ID'])

In [165]:
# setting up for k-means
x_val = df1.to_numpy()
x_val[np.isinf(x_val)] = 0
print(np.where(np.isinf(x_val)))

(array([], dtype=int64), array([], dtype=int64))


In [166]:
# getting predicted clusters of validation data points & cluster frequency
val_cluster_pred = kmeans.predict(x_val).astype(int)
temp = np.unique(val_cluster_pred, return_counts=True)
counts_val = dict(zip(temp[0], temp[1]))
counts_val.setdefault(' ', 0)
print(counts_val)

{0: 53160, 2: 62558, 4: 47169, 6: 40476, 7: 54775, 8: 29149, 9: 46701, ' ': 0}


In [167]:
print(set(val_cluster_pred))

{0, 2, 4, 6, 7, 8, 9}


In [168]:
# generating sample weights
counts_val[3] = 0
counts_val[1] = 0
counts_val[5] = 0
weights = x.shape[0] * [0]
for index in range (x.shape[0]):
    weights[index] = counts_val[train_cluster_pred[index]] / counts[train_cluster_pred[index]]

In [171]:
weights = np.array(weights).astype(float) * (x_val.shape[0] / x.shape[0])
weights

array([0.11874509, 0.11932827, 0.11932827, ..., 0.11874509, 0.11933428,
       0.11912006])

In [172]:
# saving sample weightsusing pickles        
with open('models/sample_weights.pickle', 'wb') as handle:
        pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)